In [32]:
!pip3 install bokeh

In [33]:
import numpy as np

In [34]:
from bokeh.io import push_notebook, show, output_notebook
from bokeh.layouts import row
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool

output_notebook()

Loading BokehJS ...

In [ ]:
import pandas as pd
fields = ['WEIGHT', 'ABOID', 'AGEGRP', 'ATTSCH', 'BedRm', 'BFNMEMB', 'CFInc',
       'CFInc_AT', 'CfSize', 'CFSTAT', 'CIP2011', 'CIP2011_STEM_SUM',
       'Citizen', 'CitOth', 'CMA', 'CONDO', 'COW', 'DETH123', 'DPGRSUM',
       'DTYPE', 'EFDecile', 'EfDIMBM', 'EFInc', 'EFInc_AT', 'EfSize',
       'ETHDER', 'GENSTAT', 'HCORENEED_IND', 'HDGREE', 'HHInc',
       'HHInc_AT', 'HHMRKINC', 'HHSIZE', 'HHTYPE',
       'HLANO', 'IMMCAT5', 'IMMSTAT',
       'LFACT', 'LICO', 'LICO_AT', 'LOC_ST_RES', 'LoLIMA', 'LoLIMB',
       'LoMBM', 'LSTWRK', 'MarStH', 'MOB1', 'Mob5',
       'MTNNO', 'NOS', 'PKID0_1', 'PKID15_24', 'PKID2_5',
       'PKID25', 'PKID6_14', 'PKIDS', 'POB', 'POBF', 'POBM', 'PR', 'PR1',
       'PR5', 'PRIHM', 'REGIND', 'REPAIR', 'ROOMS', 'Sex', 'SHELCO',
       'SSGRAD', 'Tenur', 'VisMin', 'WRKACT']

dataset = pd.read_csv("pumf-2016-selected-features-No-MTEn-English.csv", usecols=fields)
dataset.head()
# removed market income (MrkInc, TotInc, TotInAT)

In [ ]:
motherTongueDict = {}
motherTongueDict['1'] = 'No non-official languages'
motherTongueDict['2'] = 'Aboriginal languages'
motherTongueDict['3'] = 'Arabic'
motherTongueDict['4'] = 'Mandarin'
motherTongueDict['5'] = 'Cantonese'
motherTongueDict['6'] = 'Chinese languages'
motherTongueDict['7'] = 'German'
motherTongueDict['8'] = 'Other European languages' #'Other Germanic languages'
motherTongueDict['9'] = 'Other European languages' #'Greek'
motherTongueDict['10'] = 'Urdu'
motherTongueDict['11'] = 'Persian (Farsi)'
motherTongueDict['12'] = 'Other Indo-Iranian languages'
motherTongueDict['13'] = 'Italian'
motherTongueDict['14'] = 'Polish'
motherTongueDict['15'] = 'Portuguese'
motherTongueDict['16'] = 'Punjabi (Panjabi)'
motherTongueDict['17'] = 'Spanish'
motherTongueDict['18'] = 'Slavic languages' #'Ukrainian'
motherTongueDict['19'] = 'Austro-Asiatic languages' #'Vietnamese'
motherTongueDict['20'] = 'Austro-Asiatic languages'
motherTongueDict['21'] = 'Other European languages'
motherTongueDict['22'] = 'Russian'
motherTongueDict['23'] = 'Slavic languages'
motherTongueDict['24'] = 'Other European languages' #'Uralic languages'
motherTongueDict['25'] = 'Other Afro-Asiatic and African languages' #'Other Afro-Asiatic languages'
motherTongueDict['26'] = 'Tamil'
motherTongueDict['27'] = 'All other languages' #'Other Dravidian languages'
motherTongueDict['28'] = 'Other East and Southeast Asian languages' #'Korean'
motherTongueDict['29'] = 'Other East and Southeast Asian languages'
motherTongueDict['30'] = 'Tagalog (Pilipino, Filipino)'
motherTongueDict['31'] = 'Other Afro-Asiatic and African languages' #'Niger-Congo languages and other African languages'
motherTongueDict['32'] = 'All other languages' #'All other single languages'
motherTongueDict['88'] = 'Not available'

In [ ]:
homeLanguageDict = {}
homeLanguageDict['1'] = 'No non-official languages'
homeLanguageDict['2'] = 'Aboriginal languages'
homeLanguageDict['3'] = 'Italian'
homeLanguageDict['4'] = 'Spanish'
homeLanguageDict['5'] = 'Portugese'
homeLanguageDict['6'] = 'German'
homeLanguageDict['7'] = 'Russian'
homeLanguageDict['8'] = 'Polish'
homeLanguageDict['9'] = 'Slavic languages'
homeLanguageDict['10'] = 'Other European languages'
homeLanguageDict['11'] = 'Arabic'
homeLanguageDict['12'] = 'Other Afro-Asiatic and African languages'
homeLanguageDict['13'] = 'Punjabi (Panjabi)'
homeLanguageDict['14'] = 'Urdu'
homeLanguageDict['15'] = 'Persian (Farsi)'
homeLanguageDict['16'] = 'Other Indo-Iranian languages'
homeLanguageDict['17'] = 'Cantonese'
homeLanguageDict['18'] = 'Mandarin'
homeLanguageDict['19'] = 'Chinese languages'
homeLanguageDict['20'] = 'Austro-Asiatic languages'
homeLanguageDict['21'] = 'Tagalog (Pilipino, Filipino)'
homeLanguageDict['22'] = 'Other East and Southeast Asian languages'
homeLanguageDict['23'] = 'Tamil'
homeLanguageDict['24'] = 'All other languages'
homeLanguageDict['88'] = 'Not available'

In [ ]:
dataset = dataset[dataset.HLANO != 88]
dataset = dataset[dataset.MTNNO != 88]
# class label: home language part A - first language write in component
homeLang = dataset['HLANO']
# class label: mother tongue part A - first language write in component
motherTongue = dataset['MTNNO']
fields.remove('HLANO')
fields.remove('MTNNO')
homeLanguageMapped = homeLang.apply(lambda x: homeLanguageDict[str(x)])
motherTongueMapped = motherTongue.apply(lambda x: motherTongueDict[str(x)])
languageShift = homeLanguageMapped.ne(motherTongueMapped)

fields.remove('WEIGHT')
x = dataset[fields]
weights = dataset["WEIGHT"]

In [ ]:
homeLanguageMapped

In [ ]:
motherTongueMapped

In [ ]:
languageShift

In [ ]:
x

In [ ]:
# discretization

def discretify(dataset, quantiles=10, target_fields=('SHELCO', 'MrkInc', 'TotInc', 'TotInc_AT', 'VALUE'),
               invalid_quantile = 88):
    """Convert target fields in dataset to quantile categories."""
    
    invalid_values = {
        'SHELCO': (),
        'MrkInc': (88888888, 99999999),
        'TotInc': (88888888, 99999999),
        'TotInc_AT': (88888888, 99999999),
        'VALUE': (88888888, 99999999)
    }
    
    dataset_copy = dataset.copy()

    for continuous in target_fields:
        if continuous in dataset_copy:
            print(continuous)
            values = []
            quantile_values = pd.qcut(dataset_copy[continuous].rank(method='first'), quantiles, False)

            for value, quantile in zip(dataset_copy[continuous], quantile_values):
                if value in invalid_values[continuous]:
                    values.append(invalid_quantile)
                else:
                    values.append(quantile)

            dataset_copy[continuous] = values
    
    return dataset_copy


x = discretify(x)

x

In [ ]:
weights

In [ ]:
!pip3 install sklearn

In [ ]:
import sys 
sys.version

In [ ]:
from sklearn import datasets

In [ ]:
from sklearn.naive_bayes import CategoricalNB
classifier = CategoricalNB()

In [ ]:
from sklearn.model_selection import train_test_split
weights_train, weights_test, x_train, x_test, languageShift_train, languageShift_test = train_test_split(weights, x, languageShift, random_state=42)

In [ ]:
x_train[(x_train < 0).any(1)]

In [ ]:
classifier.fit(x_train, languageShift_train, sample_weight=weights_train.tolist())

In [ ]:
languageShift_pred = classifier.predict(x_test)

languageShift_probs = classifier.predict_proba(x_test)[:, 1]

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

The Confusion Matrix indicates, given the predicted language (row names), for how many records did the algorith match the actual language. The classification report and accuracy score are other ways to evaluate how well the classification algorithm performed.

In [ ]:
confusionMatrixLanguageShift = pd.DataFrame(
    confusion_matrix(languageShift_test,languageShift_pred), 
    index=['False', 'True'], 
    columns=['False', 'True'], 
)

In [ ]:
print(confusionMatrixLanguageShift)
print(classification_report(languageShift_test,languageShift_pred,target_names=['False', 'True']))
print(accuracy_score(languageShift_test, languageShift_pred))

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve

auc_score = roc_auc_score(languageShift_test, languageShift_probs)
auc_score

In [ ]:
base_fpr, base_tpr, _ = roc_curve(languageShift_test, [1 for _ in range(len(languageShift_test))])
model_fpr, model_tpr, _ = roc_curve(languageShift_test, languageShift_probs)

# Generate a figure container
plot = figure(plot_height=400,
              plot_width=400,
              title='ROC AUC Score',
              x_axis_label='False Positive Rate',
              y_axis_label='True Positive Rate',
              x_range=[0, 1],
              y_range=[0, 1])

plot.line(model_fpr, model_tpr, line_width=3, color='red', legend_label='Naive Bayes')
plot.line(base_fpr, base_tpr, line_width=3, line_dash='dashed', legend_label='Random')

plot.title.align = 'center'
plot.legend.location = 'bottom_right'

show(plot)

In [ ]:
!pip install tabulate

In [ ]:
classifier.classes_

In [ ]:
from tabulate import tabulate
importances = classifier.feature_log_prob_
print(importances)
#headers = ["name", "prob of feature given class: False", "prob of feature given class: True"]
#values = sorted(zip(x_train.columns, np.exp(importances[0]), np.exp(importances[1])), key=lambda x: (x[2] * -1, x[1] * -1))
#print(tabulate(values, headers, tablefmt="plain"))

In [ ]:
!pip install bokeh

In [ ]:
#from sklearn.tree import export_graphviz
#estimator = classifier.estimators_[5]
# Creates dot file named randomForestTreeSample.dot
#export_graphviz(estimator, 
#                out_file='randomForestTreeSample.dot', 
#                feature_names = list(x.columns),
#                class_names = ['False', 'True'],
#                rounded = True, proportion = False, 
#                precision = 2, filled = True,
#                max_depth=5)